# Exercise 1: Quality Control in Manufacturing using Fuzzy Systems and Expert Rules

### Librerias a utilizar

In [ ]:
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl
import matplotlib.pyplot as plt

### Definir las variables de entrada y salida

'dimensions' representa el tamaño del producto (Small, Medium, Large)

'weight' representa el peso del producto (Light, Medium, Heavy)

'material_type' indica si el material es de grado normal o alto

'surface_finish' indica si el acabado es rugoso o liso

'quality' es la salida que representa la calidad del producto

In [ ]:
dimensions = ctrl.Antecedent(np.arange(0, 101, 1), 'dimensions')
weight = ctrl.Antecedent(np.arange(0, 51, 1), 'weight')
material_type = ctrl.Antecedent(np.arange(0, 2, 1), 'material_type')  # 0: Normal, 1: High-Grade
surface_finish = ctrl.Antecedent(np.arange(0, 2, 1), 'surface_finish')  # 0: Rough, 1: Smooth
quality = ctrl.Consequent(np.arange(0, 101, 1), 'quality')

### Definir funciones de membresía para cada variable de entrada y salida

In [ ]:
dimensions['Small'] = fuzz.trimf(dimensions.universe, [0, 0, 50])
dimensions['Medium'] = fuzz.trimf(dimensions.universe, [30, 50, 70])
dimensions['Large'] = fuzz.trimf(dimensions.universe, [50, 100, 100])

weight['Light'] = fuzz.trimf(weight.universe, [0, 0, 25])
weight['Medium'] = fuzz.trimf(weight.universe, [10, 25, 40])
weight['Heavy'] = fuzz.trimf(weight.universe, [25, 50, 50])

material_type['Normal'] = fuzz.trimf(material_type.universe, [0, 0, 1])
material_type['High-Grade'] = fuzz.trimf(material_type.universe, [0, 1, 1])

surface_finish['Rough'] = fuzz.trimf(surface_finish.universe, [0, 0, 1])
surface_finish['Smooth'] = fuzz.trimf(surface_finish.universe, [0, 1, 1])

quality['Low'] = fuzz.trimf(quality.universe, [0, 0, 50])
quality['Medium'] = fuzz.trimf(quality.universe, [25, 50, 75])
quality['High'] = fuzz.trimf(quality.universe, [50, 100, 100])

### Definir reglas difusas basadas en las condiciones especificadas

In [ ]:
rule1 = ctrl.Rule(dimensions['Small'] & weight['Light'], quality['Low'])
rule2 = ctrl.Rule(dimensions['Medium'] & weight['Medium'], quality['Medium'])
rule3 = ctrl.Rule(dimensions['Large'] & weight['Heavy'], quality['High'])
rule4 = ctrl.Rule(material_type['High-Grade'] & surface_finish['Smooth'], quality['High'])

### Crear el sistema de control difuso para la calidad del producto

In [ ]:
quality_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4])
quality_sim = ctrl.ControlSystemSimulation(quality_ctrl)

### Simulación del sistema con valores de entrada

In [ ]:
dimension_input = 60  # Ejemplo de dimensión del producto
weight_input = 30  # Ejemplo de peso del producto
material_type_input = 1  # High-Grade
surface_finish_input = 1  # Smooth

### Asignar valores de entrada al sistema

In [ ]:
quality_sim.input['dimensions'] = dimension_input
quality_sim.input['weight'] = weight_input
quality_sim.input['material_type'] = material_type_input
quality_sim.input['surface_finish'] = surface_finish_input

### Ejecutar el sistema difuso

In [ ]:
quality_sim.compute()

### Determinar la categoría de calidad con base en el puntaje obtenido

In [ ]:
quality_score = quality_sim.output['quality']
if quality_score <= 33:
    quality_category = "Low"
elif quality_score <= 66:
    quality_category = "Medium"
else:
    quality_category = "High"

# Mostrar la categoría de calidad
print(quality_category)

### Visualización de funciones de membresía

In [ ]:
# Visualización de funciones de membresía
fig, axes = plt.subplots(3, 1, figsize=(8, 12))

axes[0].plot(dimensions.universe, dimensions['Small'].mf, label='Small')
axes[0].plot(dimensions.universe, dimensions['Medium'].mf, label='Medium')
axes[0].plot(dimensions.universe, dimensions['Large'].mf, label='Large')
axes[0].set_title('Dimensions')
axes[0].legend()

axes[1].plot(weight.universe, weight['Light'].mf, label='Light')
axes[1].plot(weight.universe, weight['Medium'].mf, label='Medium')
axes[1].plot(weight.universe, weight['Heavy'].mf, label='Heavy')
axes[1].set_title('Weight')
axes[1].legend()

axes[2].plot(quality.universe, quality['Low'].mf, label='Low')
axes[2].plot(quality.universe, quality['Medium'].mf, label='Medium')
axes[2].plot(quality.universe, quality['High'].mf, label='High')
axes[2].set_title('Quality')
axes[2].legend()

plt.tight_layout()
plt.show()


# Exercise 2: Driver Assistance System using Bayesian Networks and Expert Rules

This report presents a driver assistance system that uses Bayesian networks to predict the risk of collision based on the vehicle's speed, distance to the vehicle ahead, and road condition. Expert rules are integrated to provide specific recommendations to the driver.

### Structure of the Bayesian Network

The Bayesian network consists of the following nodes:

Speed: Vehicle speed (High, Medium, Low).

Distance: Distance to the vehicle ahead (Short, Medium, Long).

RoadCondition: Road condition (Wet, Dry).

CollisionRisk: Risk of collision (High, Medium, Low).

The structure of the network is as follows:

Speed, Distance, and RoadCondition are parent nodes of CollisionRisk.

### Definition of Conditional Probability Tables (CPTs)

The conditional probabilities for each node are defined below.

P(Speed)------------------

Speed	Probability

High	0.3

Medium	0.5

Low	0.2

P(Distance)-----------------------


Distance	Probability

Short	0.4

Medium	0.4

Long	0.2

P(RoadCondition)-------------------

Road Condition	Probability

Wet	0.3

Dry	0.7

P(CollisionRisk | Speed, Distance, RoadCondition) ---------------------

This table defines the probability of collision risk given the speed, distance, and road condition.

### Import required libraries

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

### Define the structure of the Bayesian network

In [ ]:
model = BayesianNetwork([
    ('Speed', 'CollisionRisk'),
    ('Distance', 'CollisionRisk'),
    ('RoadCondition', 'CollisionRisk')
])

### Define Conditional Probability Tables (CPTs)

In [ ]:
cpd_speed = TabularCPD(variable='Speed', variable_card=3, values=[[0.3], [0.5], [0.2]],
                       state_names={'Speed': ['High', 'Medium', 'Low']})

cpd_distance = TabularCPD(variable='Distance', variable_card=3, values=[[0.4], [0.4], [0.2]],
                          state_names={'Distance': ['Short', 'Medium', 'Long']})

cpd_road = TabularCPD(variable='RoadCondition', variable_card=2, values=[[0.3], [0.7]],
                      state_names={'RoadCondition': ['Wet', 'Dry']})

cpd_collision = TabularCPD(variable='CollisionRisk', variable_card=3,
                           values=[
                               [0.9, 0.7, 0.6, 0.4, 0.3, 0.1, 0.7, 0.5, 0.4, 0.2, 0.1, 0.05, 0.5, 0.3, 0.2, 0.1, 0.05, 0.01],
                               [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
                               [0.0, 0.1, 0.1, 0.2, 0.2, 0.3, 0.1, 0.2, 0.2, 0.3, 0.3, 0.25, 0.2, 0.3, 0.3, 0.3, 0.25, 0.19]
                           ],
                           evidence=['Speed', 'Distance', 'RoadCondition'],
                           evidence_card=[3, 3, 2],
                           state_names={'CollisionRisk': ['High', 'Medium', 'Low'],
                                        'Speed': ['High', 'Medium', 'Low'],
                                        'Distance': ['Short', 'Medium', 'Long'],
                                        'RoadCondition': ['Wet', 'Dry']})

### Add CPTs to the model

In [ ]:
model.add_cpds(cpd_speed, cpd_distance, cpd_road, cpd_collision)

# Validate the model
assert model.check_model()

# Perform inference using Variable Elimination
inference = VariableElimination(model)

### Function to get recommendations based on collision risk

In [ ]:
def get_recommendation(collision_risk):
    if collision_risk == 'High':
        return "High collision risk! Immediately reduce speed, increase distance from the vehicle ahead, and drive defensively."
    elif collision_risk == 'Medium':
        return "Moderate collision risk. Reduce speed and increase distance from the vehicle ahead."
    elif collision_risk == 'Low':
        return "Low collision risk. Maintain safe driving practices and stay alert to traffic conditions."
    else:
        return "Collision risk could not be determined."

### Function to simulate the driver assistance system

In [ ]:
def driver_assistance_system(speed, distance, road_condition):
    # Calculate collision risk
    result = inference.query(variables=['CollisionRisk'], evidence={'Speed': speed, 'Distance': distance, 'RoadCondition': road_condition})
    collision_risk = result.values.argmax()  # Get the state with the highest probability
    collision_risk_state = model.get_cpds('CollisionRisk').state_names['CollisionRisk'][collision_risk]
    
    # Get recommendation
    recommendation = get_recommendation(collision_risk_state)
    
    # Display results
    print(f"Calculated Collision Risk: {collision_risk_state}")
    print(f"Recommendation: {recommendation}")

### Simulate different scenarios

In [ ]:
print("Scenario 1: High speed, short distance, wet road")
driver_assistance_system(speed='High', distance='Short', road_condition='Wet')

In [ ]:
print("\nScenario 2: Medium speed, medium distance, dry road")
driver_assistance_system(speed='Medium', distance='Medium', road_condition='Dry')

In [ ]:
print("\nScenario 3: Low speed, long distance, wet road")
driver_assistance_system(speed='Low', distance='Long', road_condition='Wet')

### Discussion

The combination of Bayesian networks and expert rules provides a robust framework for real-time decision-making in driver assistance systems:

Bayesian Networks: Handle uncertainty and probabilistic reasoning based on observed data.

Expert Rules: Incorporate domain knowledge and additional factors (e.g., driver behavior, vehicle type) to refine predictions.

This system enhances driver safety by providing real-time risk assessments and enabling proactive measures to prevent collisions.

### Conclusion
This driver assistance system demonstrates the effectiveness of Bayesian networks in predicting collision risks and providing actionable recommendations. It can be extended to include more variables and rules for further improvements.